# Smart Pension - FX API demo notebook

In [1]:
# Import modules for demo notebook
import os
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# Import FX class from custom fx_api package
from fx_api import FX

# fx_api

I have designed the API with the goal of obtaining any combination of exchange rates from exchangeratesapi.io in two steps:

1. Initialise instance of FX class with desired mapping from source to target currencies.
2. Apply methods to instance to obtain exchange rates in desired date range

In [3]:
# Demo of class docstrings
help(FX)

Help on class FX in module fx_api.api:

class FX(builtins.object)
 |  FX(source_currency='GBP', target_currency=None)
 |  
 |  FX class allows user to specify source and target currencies when initialising an instance. Various methods can then be applied to obtain desired exchange rates.
 |  
 |  Class Attributes
 |  -------
 |  base_url : str
 |      URL of exchangeratesapi.io
 |  limits_calls : int, optional
 |      number of calls permitted in limits_period before rate-limiting begins, by default 20
 |  limits_period : int, optional
 |      number of seconds before limits_calls count is reset, by default 20
 |  limits_max_tries : int, optional
 |      number of tries before exponential backoff ends and returns RateLimitException, by default 10
 |  
 |  Instance Attributes
 |  -------
 |  source_currency: str or list of str
 |      currency which is to be converted
 |  target_currency: str or list of str or None
 |      target for currency being converted
 |  
 |  Methods
 |  -------

## Query demos

In [4]:
# Demo of default mapping of source_currency = "GBP" to all target_currencies for latest exchange rates
# N.B. exchange rates are only updated on working days. Therefore querying on non-working days will return 
# exchange rates from latest working day
FX_df = FX().get_FX_latest()
print(FX_df)

          date source_currency target_currency  exchange_rate_to_target
0   2021-03-12             GBP             AUD                 1.793441
1   2021-03-12             GBP             BGN                 2.278558
2   2021-03-12             GBP             BRL                 7.738219
3   2021-03-12             GBP             CAD                 1.744626
4   2021-03-12             GBP             CHF                 1.292480
5   2021-03-12             GBP             CNY                 9.044795
6   2021-03-12             GBP             CZK                30.465428
7   2021-03-12             GBP             DKK                 8.663832
8   2021-03-12             GBP             EUR                 1.165026
9   2021-03-12             GBP             GBP                 1.000000
10  2021-03-12             GBP             HKD                10.793033
11  2021-03-12             GBP             HRK                 8.840799
12  2021-03-12             GBP             HUF               426

In [5]:
# Demo of mapping of source_currency = ["GBP", "USD", "CAD", "EUR"] to target_currency = ["JPY", "AUD"] 
# for exchange rates as at date "2021-03-10"
FX_df = FX(
    source_currency = ["GBP", "USD", "CAD", "EUR"],
    target_currency = ["JPY", "AUD"]
).get_FX_date(
    date = "2021-03-10"
)
print(FX_df)

         date source_currency target_currency  exchange_rate_to_target
0  2021-03-10             CAD             AUD                 1.025863
1  2021-03-10             CAD             JPY                85.845356
2  2021-03-10             EUR             AUD                 1.543000
3  2021-03-10             EUR             JPY               129.120000
4  2021-03-10             GBP             AUD                 1.801413
5  2021-03-10             GBP             JPY               150.744265
6  2021-03-10             USD             AUD                 1.297511
7  2021-03-10             USD             JPY               108.577195


In [6]:
# Demo of mapping of source_currency = "USD" to target_currency = "GBP"
# for exchange rates as between dates "2021-02-10" and "2021-03-10"
FX_df = FX(
    source_currency = "USD",
    target_currency = "GBP"
).get_FX_date_range(
    start_at = "2021-02-10",
    end_at = "2021-03-10"
)
print(FX_df)

          date source_currency target_currency  exchange_rate_to_target
0   2021-02-10             USD             GBP                 0.722767
1   2021-02-11             USD             GBP                 0.722442
2   2021-02-12             USD             GBP                 0.724752
3   2021-02-15             USD             GBP                 0.719144
4   2021-02-16             USD             GBP                 0.719204
5   2021-02-17             USD             GBP                 0.721061
6   2021-02-18             USD             GBP                 0.716154
7   2021-02-19             USD             GBP                 0.712645
8   2021-02-22             USD             GBP                 0.713179
9   2021-02-23             USD             GBP                 0.710763
10  2021-02-24             USD             GBP                 0.708299
11  2021-02-25             USD             GBP                 0.706814
12  2021-02-26             USD             GBP                 0

# API demonstration exercise

In this exercise, you will need to query an api (https://exchangeratesapi.io/). You will need to:

1. Query the api for exchange rates for the period 01/01/2018 - 01/01/2021

2. For each day, retrieve the exchange rates for UK Sterling (GBP), US Dollars (USD) and Euros (EUR) and the conversion rates to all currencies provided by the API

3. Output the collected data to a CSV file

4. Upload the CSV file to a database (any local database will be sufficient, e.g. SQLLite, MySQL, etc.)

In [7]:
# Create filepaths for CSV & SQLLite database
script_location = os.path.abspath('')
csv_path = os.path.join(script_location, "exchange_rates.csv")
sqlite_path = os.path.join(script_location, "exchange_rates.sqlite3")

In [8]:
# Steps 1 & 2: Query API
FX_df = FX(
    source_currency=["GBP", "USD", "EUR"]
).get_FX_date_range(
    start_at="2018-01-01", 
    end_at="2021-01-01"
)
print(FX_df.head(20))

          date source_currency target_currency  exchange_rate_to_target
0   2018-01-02             EUR             AUD                  1.54130
1   2018-01-02             EUR             BGN                  1.95580
2   2018-01-02             EUR             BRL                  3.95040
3   2018-01-02             EUR             CAD                  1.51280
4   2018-01-02             EUR             CHF                  1.17180
5   2018-01-02             EUR             CNY                  7.83380
6   2018-01-02             EUR             CZK                 25.49400
7   2018-01-02             EUR             DKK                  7.44370
8   2018-01-02             EUR             GBP                  0.88953
9   2018-01-02             EUR             HKD                  9.42830
10  2018-01-02             EUR             HRK                  7.46400
11  2018-01-02             EUR             HUF                308.59000
12  2018-01-02             EUR             IDR              1626

In [9]:
# Step 3: Output data to CSV
FX_df.to_csv(
    csv_path,
    index=False
)

In [10]:
# Step 4: Upload CSV to SQLLite database

# Import CSV
csv_FX_df = pd.read_csv(
    csv_path
)

# Create exchange_rates table
# (N.B. dropping and creating each time for demo purposes)
sqlite3_con = sqlite3.connect(sqlite_path)
cur = sqlite3_con.cursor()

drop_existing_exchange_rates_sql = """
DROP TABLE IF EXISTS exchange_rates
"""
cur.execute(drop_existing_exchange_rates_sql)

exchange_rates_sql = """
CREATE TABLE exchange_rates (
    id integer PRIMARY KEY,
    date text NOT NULL,
    source_currency text NOT NULL,
    target_currency text NOT NULL,
    exchange_rate_to_target real NOT NULL
);
"""
cur.execute(exchange_rates_sql)

# Upload pandas DataFrame using SQLAlchemy
engine = create_engine(
    f"sqlite:///{sqlite_path}"
)
sqlalchemy_con = engine.connect()

csv_FX_df.to_sql(
    "exchange_rates",
    con=sqlalchemy_con,
    if_exists="append",
    index=False
)

# Query exchange_rates table to verify successful upload
exchange_rate_query_sql = """
SELECT *
FROM exchange_rates
"""
sql_FX_df = pd.read_sql(
    exchange_rate_query_sql,
    con=sqlite3_con
)
print(sql_FX_df.head(20))

    id        date source_currency target_currency  exchange_rate_to_target
0    1  2018-01-02             EUR             AUD                  1.54130
1    2  2018-01-02             EUR             BGN                  1.95580
2    3  2018-01-02             EUR             BRL                  3.95040
3    4  2018-01-02             EUR             CAD                  1.51280
4    5  2018-01-02             EUR             CHF                  1.17180
5    6  2018-01-02             EUR             CNY                  7.83380
6    7  2018-01-02             EUR             CZK                 25.49400
7    8  2018-01-02             EUR             DKK                  7.44370
8    9  2018-01-02             EUR             GBP                  0.88953
9   10  2018-01-02             EUR             HKD                  9.42830
10  11  2018-01-02             EUR             HRK                  7.46400
11  12  2018-01-02             EUR             HUF                308.59000
12  13  2018

# Additional considerations

## Automated unit tests

Unit tests implemented using pytest module and can be found in tests/ folder

From root directory of project open shell and run:
```
pytest
```

## API rate-limiting

exchangeratesapi.io does not implement rate-limiting, however, as a proof-of-concept I have added a default limit of 20 calls in 20 seconds. If the rate-limit is exceeded then the API will attempt exponential backoff a default of 10 times before raising RateLimitException

In [11]:
# Check default rate-limiting params
print(f"limits_calls: {FX.limits_calls}")
print(f"limits_period: {FX.limits_period}")
print(f"limits_max_tries: {FX.limits_max_tries}")

limits_calls: 20
limits_period: 20
limits_max_tries: 10


In [13]:
# Query API 25 times successively and observe the first 20 complete immediately, 
# at which point the API initiates rate-limiting and attempts exponential backoff until limits_period is exceeded
for i in range(25):
    FX().get_FX_latest()
    print(i+1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


## API tokens/secrets

exchangeratesapi.io does not require an API token or any passwords, however, for APIs that do require these (e.g. Azure Client) the simplest methods are to either store them in system environment variables and access with os.environ or store them in a config file (txt/ini/py/etc) that is included in the .gitignore and therefore not included in the source code

## High volume performance

API capable of downloading latest FX in <0.5s and can obtain all FX from "1999-01-04" to "2021-01-01" (~500,000 rows) in ~25-30s

In [14]:
%timeit FX(source_currency=["GBP", "USD", "EUR"]).get_FX_latest()

372 ms ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%time FX(source_currency=["GBP", "USD", "EUR"]).get_FX_date_range(start_at="1999-01-04", end_at="2021-01-01")

CPU times: user 24 s, sys: 373 ms, total: 24.4 s
Wall time: 28.3 s


,date,source_currency,target_currency,exchange_rate_to_target
0,1999-01-04,EUR,AUD,1.910000
1,1999-01-04,EUR,CAD,1.800400
2,1999-01-04,EUR,CHF,1.616800
3,1999-01-04,EUR,CYP,0.582310
4,1999-01-04,EUR,CZK,35.107000
...,...,...,...,...
540099,2020-12-31,USD,SGD,1.321653
540100,2020-12-31,USD,THB,29.929916
540101,2020-12-31,USD,TRY,7.426534
540102,2020-12-31,USD,USD,1.000000


## Error handling demos

API raises an error when either supplied incorrect parameters or an HTTP error code is returned.
This can then be logged and handled in try/except blocks accordingly by the main script utilising the API

In [16]:
# Incorrect initialisation parameter typings
FX_df = FX(
    source_currency = 4,
    target_currency = [1,"USD"]
).get_FX_latest()

TypeError: source_currency must be a string or a list of strings

In [17]:
# Incorrect date format (expects YYYY-MM-DD)
FX_df = FX().get_FX_date(date="2020-12-123")

ValueError: date must be provided in the date format YYYY-MM-DD

In [18]:
# HTTP exception raised when non-existant class supplied
FX_df = FX(
    source_currency="asdf"
).get_FX_latest()

HTTPError: 400 Client Error: Bad Request for url: https://api.exchangeratesapi.io/latest?base=asdf